In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
import seaborn as sns
from gensim.utils import simple_preprocess
import numpy as np
import gensim.parsing.preprocessing as pp
from gensim.models import Word2Vec, KeyedVectors
import gensim
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Постановка задачи и загрузка данных

В рамках практики по обработке текста поработаем с реальными данными, содержательно близкими к данным с первой практики, - из Конструктора ОП. В этот раз поработаем с описанием дисциплин с помощью языковых моделей.


Задача на практику - предобработать датасет:
1. получить текстовое описание каждой дисциплины датасета
2. выполнить предобработку текстовых данных
3. построить для дисциплины вектор на основе word2vec
4. потестировать текстовые представления

Задач не много, но почти все затратные по времени.

In [ ]:
!gdown 1onuImgIOYhiqwnBuFsDnvepD2R5Gkebi

Downloading...
From (original): https://drive.google.com/uc?id=1onuImgIOYhiqwnBuFsDnvepD2R5Gkebi
From (redirected): https://drive.google.com/uc?id=1onuImgIOYhiqwnBuFsDnvepD2R5Gkebi&confirm=t&uuid=2f08fbe9-7cef-46d1-9cca-6d1bfc939ff4
To: /content/df_up_courses_202308.csv
 29% 554M/1.88G [00:04<00:10, 127MB/s]Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/gdown/cli.py", line 169, in main
    download(
  File "/usr/local/lib/python3.10/dist-packages/gdown/download.py", line 325, in download
    for chunk in res.iter_content(chunk_size=CHUNK_SIZE):
  File "/usr/local/lib/python3.10/dist-packages/requests/models.py", line 816, in generate
    yield from self.raw.stream(chunk_size, decode_content=True)
  File "/usr/local/lib/python3.10/dist-packages/urllib3/response.py", line 936, in stream
    data = self.read(amt=amt, decode_content=decode_content)
  File "/usr/local/lib/python3.10/d

Из информации о дисциплине у нас есть следующая:
1. course_id - номер дисциплины (может повторяться)
2. course_title - название дисциплины
3. description - краткое описание дисциплины
4. section - разделы дисциплины
5. topic - темы, входящие в раздел дисциплины
6. qualification - уровень образования (бакалавриат, магистратура, специалитет)
7. unit_id - айди факульета
8. up_id - айди учебного плана (образовательной программы)
9. up_title - название учебного плана (образовательной прогарммы)
10. start_year - год набора
11. module_name - название модуля, в который входит дисциплина

In [ ]:
df = pd.read_csv("drive/My Drive/data_analysis_data/lab3/df_up_courses_202308.csv")
df.head()

,Unnamed: 0,course_id,course_title,description,section,topic,qualification,unit_id,up_id,up_title,start_year,module_name
0,0,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,"Понятия ИС, КИС, АСУ, АСУП. ИС как инструмент ...",bachelor,14.0,6905,Программирование и интернет-технологии,2018,"Специализация ""Информационные системы и технол..."
1,1,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,"Понятия ИС, КИС, АСУ, АСУП. ИС как инструмент ...",bachelor,14.0,6938,Программирование и интернет-технологии,2019,"Специализация ""Информационные системы и технол..."
2,2,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,"Понятия ИС, КИС, АСУ, АСУП. ИС как инструмент ...",bachelor,14.0,7014,Программирование и интернет-технологии,2020,"Специализация ""Разработка информационных систе..."
3,54,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,Различные методики управления организацией,bachelor,14.0,6905,Программирование и интернет-технологии,2018,"Специализация ""Информационные системы и технол..."
4,55,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,Различные методики управления организацией,bachelor,14.0,6938,Программирование и интернет-технологии,2019,"Специализация ""Информационные системы и технол..."


## 3.1 Предобработка датасета

Преобразуйте датасет так, чтобы одна строка описывала одну дисциплину в учебном плане. Отдельным столбцом должен храниться весь текст, который описывает дисциплину: description, section, topic. Без дубликатов содержания разделов и тем внутри дисциплины.

### Процесс преобразования

In [ ]:
df.dropna(inplace=True)
new_rows = []

grouped = df.groupby(['course_id', 'course_title', 'qualification', 'unit_id', 'up_id', 'up_title', 'start_year', 'module_name'])

for group_name, group_df in grouped:
    combined_description = group_df[['description', 'section', 'topic']].astype(str).agg(' '.join, axis=1)
    combined_description = combined_description.unique()[0]

    new_row = {
        'course_id': group_name[0],
        'course_title': group_name[1],
        'qualification': group_name[2],
        'unit_id': group_name[3],
        'up_id': group_name[4],
        'up_title': group_name[5],
        'start_year': group_name[6],
        'module_name': group_name[7],
        'text': combined_description
    }
    new_rows.append(new_row)

preprocessed_df = pd.DataFrame(new_rows)

KeyboardInterrupt: 

In [ ]:
preprocessed_df

### Запись преобразованных данных на гугл диск

In [ ]:
writer = open('drive/My Drive/data_analysis_data/lab3/preprocessed_df.csv', 'w', encoding='utf-8')
writer.write('')
writer.close()
preprocessed_df.to_csv("drive/My Drive/data_analysis_data/lab3/preprocessed_df.csv", index=False)

### Загрузка преобразованных данных из предыдущего блока кода ^ из `.csv` файла

In [ ]:
preprocessed_df = pd.read_csv("drive/My Drive/data_analysis_data/lab3/preprocessed_df.csv")
preprocessed_df

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text
0,2626,Философия,bachelor,8.0,6905,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Курс «Философия» даёт целостное представление ...
1,2626,Философия,bachelor,8.0,6938,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...
2,2626,Философия,bachelor,8.0,6974,Информатика и программирование,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...
3,2626,Философия,bachelor,8.0,7014,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...
4,2626,Философия,bachelor,8.0,7136,Информатика и программирование,2021,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...
...,...,...,...,...,...,...,...,...,...
65905,25731,Основы планирования научных исследований и упр...,master,63.0,7255,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (RU),Дисциплина «Основы планирования научных исслед...
65906,25732,Основы планирования научных исследований и упр...,master,63.0,7255,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (EN),"The course ""Basics of Research Planning and Ma..."
65907,25773,Дифференциальные уравнения,bachelor,51.0,7236,Фотоника и оптоинформатика,2022,Обязательные дисциплины ОП,Дифференциальные уравнения являются мощным инс...
65908,30918,Функциональная безопасность киберфизических си...,master,13.0,7201,Функциональная безопасность беспилотных трансп...,2022,Обязательные дисциплины,Целью освоения дисциплины является достижение ...


Преобразования выполняются не слишком быстро, рекомендую сохраниться.

## 3.2 Предобработка описания дисциплины

Пора поработать со столбцом с текстовым содержанием дисциплины. Минимально понадобятся следующие шаги предобработки:
1. Токенизация
2. Удаление пунктуации
3. Удаление лишних пробелов внутри текста
4. Лемматизация

Чтобы не слишком углубляться в предобработку, рекомендую воспользоваться реализованными в gensim методами. Примеры можно найти в ноутбуке по ссылке: https://colab.research.google.com/drive/1Lxe0_dPxF5_nNC1hnHTRc01rnMmUVCXa?usp=sharing

In [ ]:
%%capture
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

morph = MorphAnalyzer()

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = stopwords.words('russian') + stopwords.words('english')

In [ ]:
def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

def remove_extra_spaces(text):
    text = re.sub(' +', ' ', text)
    return text

def lemmatize_text(text):
  words = word_tokenize(text.lower())
  words = [w for w in words if not w in stop_words]
  return [morph.parse(w)[0].normal_form for w in words]

preprocessed_df2 = preprocessed_df.copy()

punctuation = []
extra_spaces = []
lemmatized = []
for item in preprocessed_df2['text']:
    first_result = remove_punctuation(item)
    punctuation.append(first_result)

    second_result = remove_extra_spaces(first_result)
    extra_spaces.append(second_result)

    third_result = lemmatize_text(second_result)
    lemmatized.append(third_result)

new_data = {'punctuation': punctuation, 'extra_spaces': extra_spaces, 'norm': lemmatized }
preprocessed_df2 = preprocessed_df2.assign(**new_data)

In [ ]:
preprocessed_df2

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,punctuation,extra_spaces,norm
0,2626,Философия,bachelor,8.0,6905,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"[курс, философия, давать, целостный, представл..."
1,2626,Философия,bachelor,8.0,6938,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"[курс, философия, давать, целостный, представл..."
2,2626,Философия,bachelor,8.0,6974,Информатика и программирование,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"[курс, философия, давать, целостный, представл..."
3,2626,Философия,bachelor,8.0,7014,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"[курс, философия, давать, целостный, представл..."
4,2626,Философия,bachelor,8.0,7136,Информатика и программирование,2021,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"[курс, философия, давать, целостный, представл..."
...,...,...,...,...,...,...,...,...,...,...,...,...
65905,25731,Основы планирования научных исследований и упр...,master,63.0,7255,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (RU),Дисциплина «Основы планирования научных исслед...,Дисциплина Основы планирования научных исследо...,Дисциплина Основы планирования научных исследо...,"[дисциплина, основа, планирование, научный, ис..."
65906,25732,Основы планирования научных исследований и упр...,master,63.0,7255,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (EN),"The course ""Basics of Research Planning and Ma...",The course Basics of Research Planning and Man...,The course Basics of Research Planning and Man...,"[course, basics, research, planning, managemen..."
65907,25773,Дифференциальные уравнения,bachelor,51.0,7236,Фотоника и оптоинформатика,2022,Обязательные дисциплины ОП,Дифференциальные уравнения являются мощным инс...,Дифференциальные уравнения являются мощным инс...,Дифференциальные уравнения являются мощным инс...,"[дифференциальный, уравнение, являться, мощный..."
65908,30918,Функциональная безопасность киберфизических си...,master,13.0,7201,Функциональная безопасность беспилотных трансп...,2022,Обязательные дисциплины,Целью освоения дисциплины является достижение ...,Целью освоения дисциплины является достижение ...,Целью освоения дисциплины является достижение ...,"[цель, освоение, дисциплина, являться, достиже..."


### Запись преобразованных данных на гугл диск

In [ ]:
writer = open('drive/My Drive/data_analysis_data/lab3/preprocessed_df2.csv', 'w', encoding='utf-8')
writer.write('')
writer.close()
preprocessed_df2.to_csv("drive/My Drive/data_analysis_data/lab3/preprocessed_df2.csv", index=False)

### Загрузка предобработанных ранее данных ^

In [ ]:
preprocessed_df2 = pd.read_csv("drive/My Drive/data_analysis_data/lab3/preprocessed_df2.csv")
preprocessed_df2

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,punctuation,extra_spaces,norm
0,2626,Философия,bachelor,8.0,6905,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '..."
1,2626,Философия,bachelor,8.0,6938,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '..."
2,2626,Философия,bachelor,8.0,6974,Информатика и программирование,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '..."
3,2626,Философия,bachelor,8.0,7014,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '..."
4,2626,Философия,bachelor,8.0,7136,Информатика и программирование,2021,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...
65905,25731,Основы планирования научных исследований и упр...,master,63.0,7255,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (RU),Дисциплина «Основы планирования научных исслед...,Дисциплина Основы планирования научных исследо...,Дисциплина Основы планирования научных исследо...,"['дисциплина', 'основа', 'планирование', 'науч..."
65906,25732,Основы планирования научных исследований и упр...,master,63.0,7255,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (EN),"The course ""Basics of Research Planning and Ma...",The course Basics of Research Planning and Man...,The course Basics of Research Planning and Man...,"['course', 'basics', 'research', 'planning', '..."
65907,25773,Дифференциальные уравнения,bachelor,51.0,7236,Фотоника и оптоинформатика,2022,Обязательные дисциплины ОП,Дифференциальные уравнения являются мощным инс...,Дифференциальные уравнения являются мощным инс...,Дифференциальные уравнения являются мощным инс...,"['дифференциальный', 'уравнение', 'являться', ..."
65908,30918,Функциональная безопасность киберфизических си...,master,13.0,7201,Функциональная безопасность беспилотных трансп...,2022,Обязательные дисциплины,Целью освоения дисциплины является достижение ...,Целью освоения дисциплины является достижение ...,Целью освоения дисциплины является достижение ...,"['цель', 'освоение', 'дисциплина', 'являться',..."


Лемматизация выполняется сравнительно долго, поэтому нужно обязательно в этот момент сохранить датасет. Также этот датасет **понадобится для выполненения задания 4**.

## 3.3 word2vec

In [ ]:
# обучим на своих данных модель word2vec
%%time
model = Word2Vec(preprocessed_df2['norm'],  # данные
                 min_count=1,               # минимальная частота слова в корпусе
                 vector_size=300,           # размер вектора для слова
                 workers=4,                 # количество потоков
                 window=10)                 # окно, в которое попадает слово

CPU times: user 7min 12s, sys: 3.2 s, total: 7min 15s
Wall time: 4min 35s


In [ ]:
model_path = "model_path" # будет удобно сохранить модель
model.save(model_path)

In [ ]:
def embed(tokens, w2v_model):
    embedded_vector = np.zeros(w2v_model.vector_size)

    num_tokens = 0
    for token in tokens:
        if token in w2v_model.wv.index_to_key:
            embedded_vector += w2v_model.wv.get_vector(token)
            num_tokens += 1
    if num_tokens > 0:
        embedded_vector /= num_tokens
    return embedded_vector

embed(preprocessed_df2['norm'][0], model)

array([ 3.65205932e-02,  7.71842406e-04,  6.57333977e-02, -1.06940573e+00,
       -8.87203696e-01,  2.17669061e-01, -2.42101937e-01, -8.90107649e-01,
       -4.76374521e-01, -1.81626746e-01,  4.42647065e-01,  3.20652780e-01,
        4.42685000e-01,  7.31585660e-01,  1.54438806e-01,  5.96350646e-01,
       -5.93538176e-01,  4.41111925e-01,  6.09459855e-01, -1.23326416e-02,
        1.80639342e-01,  3.86180086e-02, -1.92130689e-02,  2.13421830e-01,
        2.76560803e-01,  6.19244180e-01,  3.13889310e-01, -1.45607108e-01,
        3.00595744e-01, -1.97437941e-01, -1.09045840e-01,  3.91647224e-01,
        2.44029614e-01,  3.29531841e-01, -5.15446451e-01, -1.04096227e+00,
        9.05266351e-02, -2.76573340e-01,  2.46555621e-01, -9.18343375e-03,
       -2.03097351e-01,  2.50853250e-01, -5.60710079e-01,  4.02576672e-02,
       -6.92575152e-01,  5.17084973e-01,  6.67159304e-01,  5.71207242e-01,
       -6.74618226e-01, -1.99344553e-01, -2.06696638e-01,  1.41184583e-02,
        2.19558345e-01, -

In [ ]:
preprocessed_df2["embedding"] = preprocessed_df2['norm'].apply(lambda x: embed(x, model))
preprocessed_df2

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,punctuation,extra_spaces,norm,embedding
0,2626,Философия,bachelor,8.0,6905,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...","[0.03652059320744421, 0.000771842406342028, 0...."
1,2626,Философия,bachelor,8.0,6938,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...","[0.03652059320744421, 0.000771842406342028, 0...."
2,2626,Философия,bachelor,8.0,6974,Информатика и программирование,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...","[0.03652059320744421, 0.000771842406342028, 0...."
3,2626,Философия,bachelor,8.0,7014,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...","[0.03652059320744421, 0.000771842406342028, 0...."
4,2626,Философия,bachelor,8.0,7136,Информатика и программирование,2021,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...","[0.03652059320744421, 0.000771842406342028, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65905,25731,Основы планирования научных исследований и упр...,master,63.0,7255,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (RU),Дисциплина «Основы планирования научных исслед...,Дисциплина Основы планирования научных исследо...,Дисциплина Основы планирования научных исследо...,"['дисциплина', 'основа', 'планирование', 'науч...","[-0.042821307645212386, 0.08759352301309636, 0..."
65906,25732,Основы планирования научных исследований и упр...,master,63.0,7255,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (EN),"The course ""Basics of Research Planning and Ma...",The course Basics of Research Planning and Man...,The course Basics of Research Planning and Man...,"['course', 'basics', 'research', 'planning', '...","[0.2940407962860348, -0.2943290392650622, 0.19..."
65907,25773,Дифференциальные уравнения,bachelor,51.0,7236,Фотоника и оптоинформатика,2022,Обязательные дисциплины ОП,Дифференциальные уравнения являются мощным инс...,Дифференциальные уравнения являются мощным инс...,Дифференциальные уравнения являются мощным инс...,"['дифференциальный', 'уравнение', 'являться', ...","[-0.007455254249768302, 0.11294184870392154, 0..."
65908,30918,Функциональная безопасность киберфизических си...,master,13.0,7201,Функциональная безопасность беспилотных трансп...,2022,Обязательные дисциплины,Целью освоения дисциплины является достижение ...,Целью освоения дисциплины является достижение ...,Целью освоения дисциплины является достижение ...,"['цель', 'освоение', 'дисциплина', 'являться',...","[-0.03271791265693358, 0.05459921304848446, 0...."


Не забываем сохраниться!

In [ ]:
writer = open('drive/My Drive/data_analysis_data/lab3/preprocessed_df2.csv', 'w', encoding='utf-8')
writer.write('')
writer.close()
preprocessed_df2.to_csv("drive/My Drive/data_analysis_data/lab3/preprocessed_df2.csv", index=False)
preprocessed_df2 = pd.read_csv("drive/My Drive/data_analysis_data/lab3/preprocessed_df2.csv")
preprocessed_df2

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,punctuation,extra_spaces,norm,embedding
0,2626,Философия,bachelor,8.0,6905,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 3.65205932e-02 7.71842406e-04 6.57333977e-...
1,2626,Философия,bachelor,8.0,6938,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 3.65205932e-02 7.71842406e-04 6.57333977e-...
2,2626,Философия,bachelor,8.0,6974,Информатика и программирование,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 3.65205932e-02 7.71842406e-04 6.57333977e-...
3,2626,Философия,bachelor,8.0,7014,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 3.65205932e-02 7.71842406e-04 6.57333977e-...
4,2626,Философия,bachelor,8.0,7136,Информатика и программирование,2021,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 3.65205932e-02 7.71842406e-04 6.57333977e-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65905,25731,Основы планирования научных исследований и упр...,master,63.0,7255,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (RU),Дисциплина «Основы планирования научных исслед...,Дисциплина Основы планирования научных исследо...,Дисциплина Основы планирования научных исследо...,"['дисциплина', 'основа', 'планирование', 'науч...",[-0.04282131 0.08759352 0.0570515 -1.059228...
65906,25732,Основы планирования научных исследований и упр...,master,63.0,7255,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (EN),"The course ""Basics of Research Planning and Ma...",The course Basics of Research Planning and Man...,The course Basics of Research Planning and Man...,"['course', 'basics', 'research', 'planning', '...",[ 0.2940408 -0.29432904 0.19207126 1.395453...
65907,25773,Дифференциальные уравнения,bachelor,51.0,7236,Фотоника и оптоинформатика,2022,Обязательные дисциплины ОП,Дифференциальные уравнения являются мощным инс...,Дифференциальные уравнения являются мощным инс...,Дифференциальные уравнения являются мощным инс...,"['дифференциальный', 'уравнение', 'являться', ...",[-0.00745525 0.11294185 0.05578869 -1.066775...
65908,30918,Функциональная безопасность киберфизических си...,master,13.0,7201,Функциональная безопасность беспилотных трансп...,2022,Обязательные дисциплины,Целью освоения дисциплины является достижение ...,Целью освоения дисциплины является достижение ...,Целью освоения дисциплины является достижение ...,"['цель', 'освоение', 'дисциплина', 'являться',...",[-0.03271791 0.05459921 0.07433631 -1.072149...


## 3.4 Тестирование текстовых представлений

С предобработанными данными попытаемся решить следующие задачи:

1. Найти дисциплины, которые выбиваются из учебного плана. Для этого построим эмбеддинг учебного плана (усредним эмбеддинги всех его дисциплин) и найдем дисциплины, у которых минимальное косинусное сходство с эмбеддингом учебного плана.
2. Оценить, насколько удачно будет генерировать содержание дисциплины LLM. Структура описания дисциплины не важна. Важно содержание. Сравнивать будем эмбеддинги через косинусное сходство.

### 3.4.1 Поиск "лишних" дисциплин

Постройте эмбеддинги для учебных планов по тому же принципу, по которому строили эмбеддинги для дисциплин. Затем проранжируйте все дисциплины в учебном плане по сходству с эмбеддингом плана. Выберите порог для значения сходства, ниже которого дисциплина считается "лишней".


Не нужно оценивать абсолютно все учебные планы из датасета. Можно выбрать около 10 случайных планов или тех, которые вам интересны. И оценить только их.

Обратите внимание, что типичнее для вашей выборки: малое или большое количество "лишних" дисциплин?

Проведите эксперимент: уберите из учебных планов все дисциплины, которые есть в каждом из них. Найдите "лишние" после удаления одинаковых. Оцените результат.

In [ ]:
# put your code here

### 3.4.2 Генерация содержания

В этой части работы вам предлагается повторить пайплайн дообучения модели [LLaMA](https://github.com/facebookresearch/llama) (LLM с открытым исходным кодом) из туториала: https://colab.research.google.com/drive/1tG9eqttfnqHoQqmsiacywUG9ilUhoiCk?usp=sharing#scrollTo=qL5i358PRGgu

С текстовым описанием: https://www.datacamp.com/tutorial/fine-tuning-llama-2

Если памяти или ресурсов колаба не хватит, то попробуйте без дообучения. Но результат будет хуже.

Выберите 10-15 дисциплин (случайных или интересных вам) и с помощью модели сгенерируйте возможное описание дисциплины.

Постройте эмбеддинг для полученного описания. Оцените близость полученного эмбеддинга и того, что у вас уже был с момента создания word2vec.



In [ ]:
# put your code here

### Дополнительно (необязательно)

Среди выбранных вами дисциплин найдите те, которые реализуются больше одного года. Изучите их следующим образом:

1. Оцените, насколько содержательно близки дисциплины разных лет.
2. Оцените, к дисциплине какого года ближе всего сгенерированное содержание.